In [12]:
from pathlib import Path
from typing import Dict, Iterator, Optional, Any
from enum import Enum
import sys, asyncio
import chess
import chess.pgn
import chess.engine

In [3]:
def iter_games(pgn_path: Path) -> Iterator[chess.pgn.Game]:
    """Yield games one by one from a PGN file"""
    
    if pgn_path.suffix.lower() != ".pgn":
        raise ValueError(f"Expected a .pgn file, got: {pgn_path.suffix}")
    
    with open(pgn_path, "r", encoding="utf-8", errors="replace") as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            yield game

In [4]:
def pgn_to_db(chess_games_folder: Path, c, conn):
    """Read PGN files and insert FEN positions into the database"""       

    for pgn_file in chess_games_folder.iterdir():
        for game in iter_games(pgn_file):
            board = game.board()
            fen = board.fen()
            c.execute("INSERT OR IGNORE INTO positions (fen) VALUES (?)", (fen,))

            for move in game.mainline_moves():
                try:
                    board.push(move)
                    fen = board.fen()
                    c.execute("INSERT OR IGNORE INTO positions (fen) VALUES (?)", (fen,))
                except ValueError as e:
                    print(f"Skipping illegal move in {pgn_file}:{e}")            

    conn.commit()

    c.execute("SELECT COUNT(*) from positions")
    total = c.fetchone()[0]
    print(f"{total} positions loaded.")

In [5]:
import sqlite3

conn = sqlite3.connect("positions.db")

c = conn.cursor()

c.execute("""
          CREATE TABLE IF NOT EXISTS positions (
          id INTEGER PRIMARY KEY,
          fen TEXT UNIQUE NOT NULL,
          stockfish_score REAL,
          label TEXT
        )
""")

conn.commit()

chess_games_folder = Path("./chess_games_sample")
pgn_to_db(chess_games_folder, c, conn)

conn.close()

16835 positions loaded.


In [6]:
"""
Why this cell exists:
- python-chess launches Stockfish via asyncio.subprocess_exec.
- On Windows, the Selector event loop cannot create subprocesses, it raises NotImplementedError.
- Some Jupyter kernels on Windows start with the Selector policy by default.
- Switching to WindowsProactorEventLoopPolicy enables subprocess support in this notebook.

How to use:
- Run this cell once before creating the engine.
- On macOS or Linux this does nothing and is safe.
"""

print(f"Initial Policy: {type(asyncio.get_event_loop_policy()).__name__}")
if sys.platform.startswith("win"):
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
print(f"New Policy: {type(asyncio.get_event_loop_policy()).__name__}")


Initial Policy: WindowsSelectorEventLoopPolicy
New Policy: WindowsProactorEventLoopPolicy


In [ ]:
def get_centipawn_score(fen: str, engine: chess.engine.SimpleEngine, depth: int) -> int:
    """Gives centipawn score for a fen string"""
    
    board = chess.Board(fen)
    info = engine.analyse(board=board, limit=chess.engine.Limit(depth=depth))
    
    # If there is a forced mate, score is very large
    centipawn_score = info["score"].pov(chess.WHITE).score(mate_score=100000)

    return centipawn_score

In [10]:
stockfish_executable_path = Path("./stockfish/stockfish-windows-x86-64-avx2.exe")
engine = chess.engine.SimpleEngine.popen_uci(stockfish_executable_path)
print("Engine started.")

sample_fen = "rr6/4k2p/3bpp2/pPN2bp1/NnpP4/2B1P1P1/3K1P1P/R1R5 w - - 0 31"

centipawn_score = get_centipawn_score(fen=sample_fen, engine=engine, depth=10)
print(centipawn_score)

engine.quit()
print("Engine quit cleanly.")

Engine started.
66
Engine quit cleanly.


In [13]:
class PositionLabel(Enum):
    WHITE_WINNING = 0
    WHITE_DECISIVE = 1
    WHITE_BETTER = 2
    EQUAL = 3
    BLACK_BETTER = 4
    BLACK_DECISIVE = 5
    BLACK_WINNING = 6

    # WHITE_MATE = 
    # BLACK_MATE = 

In [ ]:
def classify_centipawn_score(cp: int) -> str:
    """
    Classify a centipawn evaluation into a categorical label.
    
    To get the numerical representation of a PositionLabel, use the .value attribute.

    Mapping (centipawns → label):
        cp >= +500        → White is winning
        +500 > cp >= +300 → White has a decisive advantage
        +300 > cp >= +100 → White is better
        +100 > cp > -100  → The position is equal
        -100 >= cp > -300 → Black is better
        -300 > cp >= -500 → Black has a decisive advantage
        cp <= -500        → Black is winning

    Todo: Extremely large values (mate scores) are not explicitly handled yet.
    """

    if cp >= 500: return PositionLabel.WHITE_WINNING
    if 500 > cp >= 300: return PositionLabel.WHITE_DECISIVE
    if 300 > cp >= 100: return PositionLabel.WHITE_BETTER
    if 100 > cp > -100: return PositionLabel.EQUAL
    if -100 >= cp > -300: return PositionLabel.BLACK_BETTER
    if -300 > cp >= -500: return PositionLabel.BLACK_DECISIVE
    return PositionLabel.BLACK_WINNING